https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api

In [ ]:
! pip install dlt[duckdb]

In [ ]:
! dlt --version

In [ ]:
import dlt
from dlt.sources.helpers.rest_client.client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

@dlt.resource(name="rides")
def ny_taxi():
    client = RESTClient(
        base_url = "https://us-central1-dlthub-analytics.cloudfunctions.net",
        paginator = PageNumberPaginator(
            base_page = 1,
            total_path = None
        )
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page

pipeline = dlt.pipeline(
    pipeline_name = "ny_taxi_pipeline",
    destination = "duckdb",
    dataset_name = "ny_tax_data"
)

In [ ]:
load_info = pipeline.run(ny_taxi)
print(load_info)

In [ ]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

conn.sql("DESCRIBE").df()

In [ ]:
df = pipeline.dataset(dataset_type="default").rides.df()

df.shape

In [ ]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
        """
        select avg(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
        from rides;
        """
    )

    print(res)